## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [2]:
!nvidia-smi

Wed Apr 23 08:50:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import os
HOME = os.getcwd()
print(HOME)


/content


## Install YOLOv8

YOLOv8 can be installed in two ways - from the source and via pip. This is because it is the first iteration of YOLO to have an official package.

In [4]:
# Pip install method (recommended)

!pip install ultralytics==8.2.103 -q

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.103 🚀 Python-3.11.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 45.7/112.6 GB disk)


In [5]:
from ultralytics import YOLO

from IPython.display import display, Image

In [6]:
!pip install ultralytics flask flask-cors pyngrok opencv-python-headless

In [7]:
! pip install pyngrok

In [8]:
!ngrok authtoken 2svEDRwLhbEkDLFP65MejfiQa4i_6tP7eTqPwo8c3m4fH1JqZ


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [9]:
from pyngrok import ngrok
ngrok.kill()
public_url = ngrok.connect(5000).public_url
print(f'Public URL: {public_url}')


Public URL: https://a9ec-35-221-254-126.ngrok-free.app


In [12]:
from google.colab import drive
drive.mount('/content/drive')

# Copy model back to working directory
!cp /content/drive/MyDrive/detection_v1.pt .

# Load YOLOv8 model
from ultralytics import YOLO
model = YOLO("/content/detection_v1.pt")  # Load trained model


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!pip install numpy


In [14]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import base64
import cv2
import numpy as np
from ultralytics import YOLO
import traceback  # For detailed error logs

app = Flask(__name__)
CORS(app)


@app.route('/predict', methods=['POST'])
def predict():
    try:
        if 'image' not in request.files:
            return jsonify({'error': 'No image provided'}), 400

        file = request.files['image']

        # Read and decode the image
        image_bytes = file.read()
        nparr = np.frombuffer(image_bytes, np.uint8)
        image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        if image is None:
            raise ValueError("Image decoding failed")

        # Run YOLO model prediction
        results = model(image)

        predictions = []
        for result in results:
            boxes = result.boxes
            for box in boxes:
                pred = {
                    'class': result.names[int(box.cls[0])],
                    'confidence': float(box.conf[0]),
                    'bbox': box.xyxy[0].tolist()
                }
                predictions.append(pred)

        # Annotate and encode the image
        _, buffer = cv2.imencode('.jpg', results[0].plot())
        image_base64 = base64.b64encode(buffer).decode('utf-8')

        return jsonify({
            'predictions': predictions,
            'image': image_base64
        })

    except Exception as e:
        print("\n🔥 Error in /predict route 🔥")
        traceback.print_exc()  # This prints the full traceback in Colab logs
        return jsonify({
            'error': 'Internal server error',
            'details': str(e)
        }), 500

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit



0: 640x640 1 Grasshopper, 16.4ms
Speed: 15.9ms preprocess, 16.4ms inference, 124.1ms postprocess per image at shape (1, 3, 640, 640)


INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 08:54:58] "POST /predict HTTP/1.1" 200 -



0: 640x640 1 beetle, 72.9ms
Speed: 5.2ms preprocess, 72.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)


...

INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 09:01:21] "POST /predict HTTP/1.1" 200 -



0: 640x640 1 Blight, 144.4ms
Speed: 5.7ms preprocess, 144.4ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)


INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 09:01:46] "POST /predict HTTP/1.1" 200 -



0: 640x640 2 Blights, 122.0ms
Speed: 3.3ms preprocess, 122.0ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)


INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 09:02:10] "POST /predict HTTP/1.1" 200 -
